In [5]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train': '../datasets/pondlet_STB_HSK3_20220714_content_data_train.csv',
                                          'test': '../datasets/pondlet_STB_HSK3_20220714_content_data_test.csv'})

Using custom data configuration default-3097a79f3ab203fd


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 816.25it/s]


Dataset csv downloaded and prepared to /home/vincent0730/.cache/huggingface/datasets/csv/default-3097a79f3ab203fd/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 429.85it/s]


In [6]:
dataset["train"][0]

{'Label': 'Lv.1',
 'Length': 173,
 'content': '山姆、李強跟琪琪在宿舍大廳，他們找來貝拉，想問她要不要一起去野餐。李強：我們明天想去野餐，妳要不要去？貝拉：可是我明天想爬山，晚上想看電視。李強：要不然明天先爬山爬三十分鐘，再開始野餐，我們可以喝咖啡吃水果，然後晚上打車去看電影，好嗎？貝拉：好，我想去走一走。看電影會看多長時間呢？李強：我們看電影會看兩個小時。貝拉：好，我會準備很多好吃的東西。'}

In [20]:
from transformers import AutoTokenizer

model_name = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["content"], truncation=True)

In [9]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

100%|██████████| 1/1 [00:00<00:00, 16.54ba/s]


In [17]:
tokenized_dataset["train"][0].keys()

dict_keys(['Label', 'Length', 'content', 'input_ids', 'token_type_ids', 'attention_mask'])

In [18]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()